<h1 align=center style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
ارزش‌گذاری مشتریان
</font>
</h1>

<p dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font>
    در ادامه‌ی امور مربوط به بازاریابی، در این مرحله باید براساس ارزش مشتریان برای شرکت اقدام به گروه‌بندی آن‌ها بکنیم.
    برای این کار از یک روش معروف حوزه‌ی تحلیل رفتار مشتری با نام <i>RFM</i>استفاده کردیم .
    در این روش برای هر مشتری سه معیار را محاسبه کردیم:
</font>
</p>


<div dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font>
    <ol>
        <li>
            <b>معیار <i>Recency</i> (ماندگاری):</b> تعداد روزی که از آخرین خرید مشتری می‌گذرد.
        </li>
        <li>
            <b>معیار <i>Frequency</i> (فرکانس):</b> تعداد دفعاتی که مشتری از ما خرید کرده است.
        </li>
        <li>
            <b>معیار <i>Monetary Value</i> (ارزش مالی):</b> مجموع درآمدی که شرکت از یک مشتری داشته است.
        </li>
    </ol>
</font>
</div>

In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
df = pd.read_csv("preprocessed_sales.csv", parse_dates=['InvoiceDate'])
df.head()

,InvoiceNumber,ProductCode,ProductName,Quantity,InvoiceDate,UnitPrice,CustomerId,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    معیار Recency
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    ابتدا نیاز است برای هر مشتری (<code>CustomerId</code>) معیار <i>Recency</i> را محاسبه بکنیم.
    برای این معیار باید تعداد روز گذشته از آخرین خرید کاربر نسبت به یک روز بعد از آخرین تاریخ موجود در مجموعه‌داده (ستون <code>InvoiceDate</code>) را محاسبه بکنیم. بخاطر این که فرض را بر آن گرفته شده که در چنین روزی در حال تحلیل هستیم.
</font>
</p>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    بنابراین ابتدا آخرین تاریخ موجود در مجموعه‌داده را پیدا کرده، و پس از جمع روز آن با <code>1</code>، حاصل را در متغیر <code>last_day</code> ذخیره کردیم.
    مقدار این متغیر باید از نوع <code>Timestamp</code> </code> باشد.
</font>
</p>

In [3]:
last_day = df['InvoiceDate'].sort_values().iloc[-1] + datetime.timedelta(days=1)
last_day

Timestamp('2010-12-10 20:01:00')

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    سپس میتوانیم به ازای هر مشتری، تاریخ آخرین خرید وی را پیدا کرده و تعداد روز گذشته از <code>last_day</code> را محاسبه بکنیم.
    <br>
    حاصل را در یک دیتافریم به نام <code>df_customer_segments</code> و با دو ستون <code>CustomerId</code> (شناسه‌ی مشتری) و <code>Recency</code> (مقدار ماندگاری محاسبه‌شده) ذخیره کردیم.
</font>
</p>

In [4]:
df.groupby('CustomerId')['InvoiceDate'].max().apply(lambda x: (last_day - x).days).index

Float64Index([12346.0, 12347.0, 12348.0, 12349.0, 12351.0, 12352.0, 12353.0,
              12355.0, 12356.0, 12357.0,
              ...
              18277.0, 18278.0, 18279.0, 18280.0, 18281.0, 18283.0, 18284.0,
              18285.0, 18286.0, 18287.0],
             dtype='float64', name='CustomerId', length=4312)

In [5]:
recency = df.groupby('CustomerId')['InvoiceDate'].max().apply(lambda x: (last_day - x).days)
df_customer_segments = pd.DataFrame({'CustomerId': recency.index, 'Recency': recency.values})
df_customer_segments.head()

,CustomerId,Recency
0,12346.0,165
1,12347.0,3
2,12348.0,74
3,12349.0,43
4,12351.0,11


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    معیار Frequency
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    اکنون باید محاسبه بکنیم هرکدام از مشتری‌ها تاکنون چند خرید یکتا داشته‌اند. حاصل را در ستونی به نام <code>Frequency</code> به دیتافریم <code>df_customer_segments</code> اضافه کردیم.
</font>
</p>

In [6]:
df_customer_segments['Frequency'] = df.groupby(['CustomerId'])['InvoiceNumber'].nunique().values
df_customer_segments.head()

,CustomerId,Recency,Frequency
0,12346.0,165,11
1,12347.0,3,2
2,12348.0,74,1
3,12349.0,43,3
4,12351.0,11,1


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    معیار Monetary Value
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    برای این معیار باید محاسبه بکنیم مجموع خرید هر مشتری چقدر بوده است؟  برای محاسبه‌ی دریافتی هر ردیف باید قیمت واحد (<code>UnitPrice</code>) را در تعداد آن (<code>Quantity</code>) ضرب بکنیم و از حاصل آن برای محاسبه‌ی مجموع خرید هر مشتری استفاده بکنیم.
     حاصل را در ستونی به نام <code>MonetaryValue</code> به دیتافریم <code>df_customer_segments</code> اضافه بکنیم.
</font>
</p>

In [7]:
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]
df_customer_segments['MonetaryValue'] = df.groupby('CustomerId')['TotalPrice'].sum().values
df_customer_segments.head()

,CustomerId,Recency,Frequency,MonetaryValue
0,12346.0,165,11,372.86
1,12347.0,3,2,1323.32
2,12348.0,74,1,222.16
3,12349.0,43,3,2671.14
4,12351.0,11,1,300.93


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    گروه‌بندی هر معیار
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    اکنون نیاز است هرکدام از این سه معیار را براساس چارک اول،‌ دوم و سوم به شکل زیر به ۴ گروه مختلف تقسیم بکنیم
</font>
</p>


<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    به ازای هر کدام از این سه معیار چنین کاری را انجام دادیم و نتایج را در سه ستون دیگر به نام‌های
    <code>F_quartile</code>، <code>R_quartile</code> و <code>M_quartile</code>
    ذخیره کردیم.
       مقادیر این ستون‌ها معادل شماره گروه یعنی اعداد <code>1</code> تا <code>4</code> هستند .
</font>
</p>

In [8]:
def get_quartile(arr):
    qu = np.quantile(arr, [0.25, 0.5, 0.75])
    output = []
    for value in arr:
        if value <= qu[0]:
            output.append(1)
        elif value > qu[0] and value <= qu[1]:
            output.append(2)
        elif value > qu[1] and value <= qu[2]:
            output.append(3)
        elif value > qu[2]:
            output.append(4)
    return output

In [9]:
df_customer_segments['R_quartile'] = get_quartile(df_customer_segments['Recency'])
df_customer_segments['F_quartile'] = get_quartile(df_customer_segments['Frequency'])
df_customer_segments['M_quartile'] = get_quartile(df_customer_segments['MonetaryValue'])
df_customer_segments.head()

,CustomerId,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile
0,12346.0,165,11,372.86,4,4,2
1,12347.0,3,2,1323.32,1,2,3
2,12348.0,74,1,222.16,3,1,1
3,12349.0,43,3,2671.14,2,3,4
4,12351.0,11,1,300.93,1,1,1


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    معیار RFM
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    حالا  با کنار هم قرار دادن مقادیر ستون‌های
    <code dir=ltr>[R_quartile, F_quartile, M_quartile]</code>
    مقدار نهایی <i>RFM</i> را محاسبه کردیم و در ستونی به همین نام ذخیره کردیم.
</font>
</p>

In [10]:
df_customer_segments['RFM'] = (df_customer_segments['R_quartile'].astype(str) +
                               df_customer_segments['F_quartile'].astype(str) +
                               df_customer_segments['M_quartile'].astype(str)).astype(int)
df_customer_segments.head()

,CustomerId,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile,RFM
0,12346.0,165,11,372.86,4,4,2,442
1,12347.0,3,2,1323.32,1,2,3,123
2,12348.0,74,1,222.16,3,1,1,311
3,12349.0,43,3,2671.14,2,3,4,234
4,12351.0,11,1,300.93,1,1,1,111


<h2 dir=rtl align=right style="line-height:200%;font-family:vazir;color:#0099cc">
<font face="vazir" color="#0099cc">
    گروه‌بندی مشتریان
</font>
</h2>

<p dir=rtl style="direction: rtl;text-align: justify;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
    حال باید براساس مقادیر ستون <code>RFM</code> و طبق جدول زیر به گروه‌بندی مشتریان بپردازیم و نام گروه آن‌ها را در ستونی به نام <code>Segment</code> ذخیره بکنیم.
</font>
</p>

</p>

In [11]:
def group_rfm(input):
    output = []
    for i in input:
        if i == 144:
            output.append('Best')
        elif i == 344:
            output.append('AlmostLost')
        elif i == 444:
            output.append('LostBigSpenders')
        elif i == 441:
            output.append('LostCheap')
        elif str(i)[1] == '4':
            output.append('Loyal')
        elif str(i)[2] == '4':
            output.append('BigSpenders')
        else:
            output.append('Normal')
    return output

In [12]:
df_customer_segments['Segment'] = group_rfm(df_customer_segments['RFM'])
df_customer_segments.head()

,CustomerId,Recency,Frequency,MonetaryValue,R_quartile,F_quartile,M_quartile,RFM,Segment
0,12346.0,165,11,372.86,4,4,2,442,Loyal
1,12347.0,3,2,1323.32,1,2,3,123,Normal
2,12348.0,74,1,222.16,3,1,1,311,Normal
3,12349.0,43,3,2671.14,2,3,4,234,BigSpenders
4,12351.0,11,1,300.93,1,1,1,111,Normal


In [13]:
import zipfile
import joblib

df_customer_segments.to_csv('df_customer_segments.csv', index=False)
joblib.dump(last_day, 'last_day')

def compress(file_names):
    print("File Paths:")
    print(file_names)
    compression = zipfile.ZIP_DEFLATED
    with zipfile.ZipFile("result.zip", mode="w") as zf:
        for file_name in file_names:
            zf.write('./' + file_name, file_name, compress_type=compression)

file_names = ["last_day", "df_customer_segments.csv", "final_project_4_rfm.ipynb"]
compress(file_names)

File Paths:
['last_day', 'df_customer_segments.csv', 'final_project_4_rfm.ipynb']
